In [5]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import time
import random
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [6]:
now = datetime.today()

In [7]:
def getcpData():
    url = "https://cpfmapi.addressasia.com/wp-json/store/v2/collect?lang=th"
    headers = {'Accept': 'application/json, text/plain, */*'}
    #form_data = {"keyword": keyword }
    #response = requests.post(url, headers=headers , data=form_data)
    response = requests.post(url, headers=headers)
    payload = response.json()
    return payload

payload = getcpData()
with open(f'../trash/Cpfreshmart{str(now.date())}.json', 'w') as f:
        json.dump(payload, f)

In [8]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try :
        road = payload['road']
    except :
        road = "" 
    try:
        postcode = payload['postcode']
        
    except :
        postcode = ""
        
    address = "-"
    
    if(road != ""):
        address = road + " " + subdistrict + " " + district + " " + province + " "  + postcode
    
    province = extracteachDetail(province)
    district = extracteachDetail(district)
    subdistrict = extracteachDetail(subdistrict)
    
    return province,district,subdistrict,postcode,road,address

In [9]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [10]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [11]:
f = open(f'../trash/Cpfreshmart{str(now.date())}.json')
payload = json.load(f)
length = len(payload)

list_cp = []

i = 0
while i < length :
    if(i == 135 and store_id == "111032"):
            payload_address = "101 ม.6 ต.ไชยสถาน อำเภอ สารภี เชียงใหม่ 50140"
            province = "เชียงใหม่"
            district = "สารภี"
            subdistrict = "ไชยสถาน"
            postcode = "50140"
    else:
        if i == 139 :
            i = i + 1
    
        name = payload[i]['name']
        store_id = payload[i]['store_id']
        store_type_name = "CP Freshmart"
        store_type = "Speciality Store"
        lat = payload[i]['latitude']
        lng = payload[i]['longitude']
        province,district,subdistrict,postcode,road,payload_address = longdoAddress(lat,lng)
    
    dict_payload = { "name": name,
                     "store_id" : store_id,
                     "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
    
    list_cp.append(dict_payload)
    print("index of: " + str(i))
    print(payload[i])
    time.sleep(1)
    
    i = i + 1

index of: 0
{'store_id': '110455', 'name': 'CP Fresh Mart- ม.หรรษา', 'latitude': '13.7042995744', 'longitude': '100.33514672', 'img': 'https://cpfmapi.addressasia.com/wp-content/plugins/cpfm-store/images/cpfm-logo-64.png', 'phone': '083-9889810'}
index of: 1
{'store_id': '111241', 'name': 'CP Fresh Mart – พุทธบูชา 36', 'latitude': '13.650022', 'longitude': '100.481251', 'img': 'https://cpfmapi.addressasia.com/wp-content/plugins/cpfm-store/images/cpfm-logo-64.png', 'phone': '083-9889845'}
index of: 2
{'store_id': '110561', 'name': 'CP Fresh Mart – ธนินทร', 'latitude': '13.94010287', 'longitude': '100.612239', 'img': 'https://cpfmapi.addressasia.com/wp-content/plugins/cpfm-store/images/cpfm-logo-64.png', 'phone': '083-9890077'}
index of: 3
{'store_id': '110933', 'name': 'CP Freshmart - สาขา ม.เกษตร', 'latitude': '13.848184980009783', 'longitude': '100.56705314567436', 'img': 'https://cpfmapi.addressasia.com/wp-content/plugins/cpfm-store/images/cpfm-logo-64.png', 'phone': '083-9890322'}
i

In [12]:
with open(f"../trash/cpfreshmart{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_cp, outfile)

In [13]:
df_json =  pd.read_json(f"../trash/cpfreshmart{str(now.date())}.json")
df_json.to_excel(f"../excel/CP/CPfreshmart{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/CP/CPfreshmart{str(now.date())}.csv")

In [ ]:
#CP
CP = pd.read_csv(f"../csv/CP/CPfreshmart{str(now.date())}.csv", header =0 )
engine = create_engine('mysql://root:@localhost:3306/cpfreshmart')
with engine.connect() as conn, conn.begin():
    CP.to_sql(f'cpfreshmart{str(now.date())}', conn, if_exists='append', index=False)
conn.close()